In [1]:
from seeq import spy
import pandas as pd

# spy.push

Uploads signals, conditions, scalars, and assets to the Seeq Server.

There are two main types of information processed by Seeq: _Data_ and _metadata_:

- **Data** is the time series and time interval information that is either collected or derived from sensor data. It consists of timestamps and values (samples), or time intervals and properties (capsules). This data can be plotted on a trend or used to train a neural network, for example.

- **Metadata** is the information about the data, that is independent of a particular point in time or time interval. For example, a signal's _name_, _description_ and _unit of measure_ is classified as metadata; or the formula that is used to derive a new signal from one or more source signals; or the asset tree that is used to model similar equipment or industrial processes.

The `spy.push()` command allows you to upload both types of information to Seeq Server. When you push _metadata_, you make entries in Seeq's data index, which allows you or other users to search for and find such entries. If you also push _data_, then samples or capsules or scalars will appear when the user selects those index entries for inclusion on a trend, scatter plot, scorecard or other visualization.

```
spy.push(data=None, metadata=None, item_type=None, workbook='Data Lab >> Data Lab Analysis',
         worksheet='From Data Lab', datasource=None, archive=False, type_mismatches='raise', errors='catalog')
```

Log into Seeq Server if you're not using Seeq Data Lab:

In [2]:
spy.login(url='http://localhost:34216', credentials_file='../credentials.key')

## Workbook scoping

When you push any type of data, it is only available/discoverable within the workbook specified by the `workbook` argument. This allows you to _sandbox_ your activity by default, and only publish to your colleagues later when your experimentation is largely over.

## Pushing data

The simplest activity you can do with the `spy.push()` command is to read in a CSV file using Pandas and push it into Seeq. It will be stored in Seeq's internal time series database.

In [3]:
import csv
csv_file = pd.read_csv('csv_import_example.csv', parse_dates=['TIME(unitless)'], index_col='TIME(unitless)')
csv_file.head()

,BITDEP(ft),BLOCKCOMP(ft),DEP_RTN(ft),DEPTH(ft),FLOWIN(USgal/min),FLOWOUTPC(%),ROP_AVG(ft/h),DIFP(psi)
TIME(unitless),,,,,,,,
2018-07-25 23:00:01-06:00,4630.217,74.813,4627.0,4630.217,799.857,40.799,142.698,454.214
2018-07-25 23:00:03-06:00,4630.354,74.735,4627.5,4630.354,799.857,40.576,142.698,481.383
2018-07-25 23:00:05-06:00,4630.470,74.619,4627.5,4630.470,799.857,39.929,142.698,487.502
2018-07-25 23:00:07-06:00,4630.588,74.501,4627.5,4630.588,799.857,40.305,210.705,561.480
2018-07-25 23:00:09-06:00,4630.699,74.390,4627.5,4630.699,799.292,40.245,210.705,610.968


When you want to push data, it must have an index with a datetime data type. That's why we used the `parse_dates` and `index_col` arguments for `Pandas.read_csv()`.

Now you can just push it into Seeq:

In [4]:
push_results = spy.push(data=csv_file)
push_results

,Count,Time,Result
BITDEP(ft),11485,00:00:02.91,Success
BLOCKCOMP(ft),11485,00:00:03.18,Success
DEP_RTN(ft),11485,00:00:03.36,Success
DEPTH(ft),11485,00:00:03.79,Success
FLOWIN(USgal/min),11485,00:00:04.07,Success
FLOWOUTPC(%),11485,00:00:04.50,Success
ROP_AVG(ft/h),11485,00:00:04.81,Success
DIFP(psi),11485,00:00:05.00,Success


,Push Count,Push Time,Push Result,ID,Type,Name
BITDEP(ft),11485.0,0:00:02.912997,Success,5F960F5E-4057-4FAD-A997-A79A4CFA39FB,StoredSignal,BITDEP(ft)
BLOCKCOMP(ft),11485.0,0:00:03.179998,Success,055C1FFA-6BF1-4FF0-8A14-C6F3EBA867C5,StoredSignal,BLOCKCOMP(ft)
DEP_RTN(ft),11485.0,0:00:03.362003,Success,BCD28750-F5E8-479A-AFA3-CB08664EE72C,StoredSignal,DEP_RTN(ft)
DEPTH(ft),11485.0,0:00:03.785000,Success,92387251-636E-4CA7-B438-B8E11143C788,StoredSignal,DEPTH(ft)
FLOWIN(USgal/min),11485.0,0:00:04.066997,Success,316D8C44-EC7B-4D1F-B9A4-98C4CE9008BB,StoredSignal,FLOWIN(USgal/min)
FLOWOUTPC(%),11485.0,0:00:04.497000,Success,23D75886-44C1-4DE8-BA8E-B854471368E2,StoredSignal,FLOWOUTPC(%)
ROP_AVG(ft/h),11485.0,0:00:04.811001,Success,AAB802BB-3F4E-4DC7-86CD-D7CA9F2F9A16,StoredSignal,ROP_AVG(ft/h)
DIFP(psi),11485.0,0:00:05.002002,Success,2B241BD7-01B8-4ADA-A0EF-BF4854EF9D86,StoredSignal,DIFP(psi)


NOTE: Pushing data can be relatively slow. This is an area that Seeq will be optimizing in future versions.

You can push multiple times, and as long as the names are the same and the workbook hasn't changed, you'll just add to the existing signal.

Now let's try pushing just metadata. You can see that the column names from the CSV file contain the unit of measure in parentheses. Let's use Pandas to extract the name and the unit of measure as separate columns.

In [5]:
better_metadata = push_results.copy()
better_metadata['Original Name'] = better_metadata.index
better_metadata['Name'] = better_metadata['Original Name'].str.extract(r'(.*)\(')
better_metadata['Value Unit Of Measure'] = better_metadata['Original Name'].str.extract(r'.*\((.*)\)')
better_metadata

,Push Count,Push Time,Push Result,ID,Type,Name,Original Name,Value Unit Of Measure
BITDEP(ft),11485.0,0:00:02.912997,Success,5F960F5E-4057-4FAD-A997-A79A4CFA39FB,StoredSignal,BITDEP,BITDEP(ft),ft
BLOCKCOMP(ft),11485.0,0:00:03.179998,Success,055C1FFA-6BF1-4FF0-8A14-C6F3EBA867C5,StoredSignal,BLOCKCOMP,BLOCKCOMP(ft),ft
DEP_RTN(ft),11485.0,0:00:03.362003,Success,BCD28750-F5E8-479A-AFA3-CB08664EE72C,StoredSignal,DEP_RTN,DEP_RTN(ft),ft
DEPTH(ft),11485.0,0:00:03.785000,Success,92387251-636E-4CA7-B438-B8E11143C788,StoredSignal,DEPTH,DEPTH(ft),ft
FLOWIN(USgal/min),11485.0,0:00:04.066997,Success,316D8C44-EC7B-4D1F-B9A4-98C4CE9008BB,StoredSignal,FLOWIN,FLOWIN(USgal/min),USgal/min
FLOWOUTPC(%),11485.0,0:00:04.497000,Success,23D75886-44C1-4DE8-BA8E-B854471368E2,StoredSignal,FLOWOUTPC,FLOWOUTPC(%),%
ROP_AVG(ft/h),11485.0,0:00:04.811001,Success,AAB802BB-3F4E-4DC7-86CD-D7CA9F2F9A16,StoredSignal,ROP_AVG,ROP_AVG(ft/h),ft/h
DIFP(psi),11485.0,0:00:05.002002,Success,2B241BD7-01B8-4ADA-A0EF-BF4854EF9D86,StoredSignal,DIFP,DIFP(psi),psi


In [6]:
spy.push(metadata=better_metadata)

,Signal,Scalar,Condition,Asset,Relationship,Overall
Items pushed,8,0,0,0,0,8


,Push Count,Push Time,ID,Type,Name,Original Name,Value Unit Of Measure,Push Result
BITDEP(ft),11485.0,0:00:02.912997,5F960F5E-4057-4FAD-A997-A79A4CFA39FB,StoredSignal,BITDEP,BITDEP(ft),ft,Success
BLOCKCOMP(ft),11485.0,0:00:03.179998,055C1FFA-6BF1-4FF0-8A14-C6F3EBA867C5,StoredSignal,BLOCKCOMP,BLOCKCOMP(ft),ft,Success
DEP_RTN(ft),11485.0,0:00:03.362003,BCD28750-F5E8-479A-AFA3-CB08664EE72C,StoredSignal,DEP_RTN,DEP_RTN(ft),ft,Success
DEPTH(ft),11485.0,0:00:03.785000,92387251-636E-4CA7-B438-B8E11143C788,StoredSignal,DEPTH,DEPTH(ft),ft,Success
FLOWIN(USgal/min),11485.0,0:00:04.066997,316D8C44-EC7B-4D1F-B9A4-98C4CE9008BB,StoredSignal,FLOWIN,FLOWIN(USgal/min),USgal/min,Success
FLOWOUTPC(%),11485.0,0:00:04.497000,23D75886-44C1-4DE8-BA8E-B854471368E2,StoredSignal,FLOWOUTPC,FLOWOUTPC(%),%,Success
ROP_AVG(ft/h),11485.0,0:00:04.811001,AAB802BB-3F4E-4DC7-86CD-D7CA9F2F9A16,StoredSignal,ROP_AVG,ROP_AVG(ft/h),ft/h,Success
DIFP(psi),11485.0,0:00:05.002002,2B241BD7-01B8-4ADA-A0EF-BF4854EF9D86,StoredSignal,DIFP,DIFP(psi),psi,Success
